In [ ]:
import pickle
def writePickle( Variable, fname):
    filename = fname +".pkl"
    f = open("pickle_vars/"+filename, 'wb')
    pickle.dump(Variable, f)
    f.close()
def readPickle(fname):
    filename = "pickle_vars/"+fname +".pkl"
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj

In [ ]:
# load splitted datasets
train_df = readPickle("train_df")
dev_df = readPickle("dev_df")
test_df = readPickle("test_df")

In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict
ID_to_POS = readPickle("final_IDs_to_POS_dict")

Artist2id = readPickle("indexing/Artist2id")

In [ ]:
def dummy_gen(database):
    dummy_train_dict = database.to_dict('list')
    sorted_dummy_train_dict = OrderedDict(sorted(dummy_train_dict.items(), key=lambda v: v, reverse=True))
    artists = list()
    songs = list()
    for artist, song_ID_list in sorted_dummy_train_dict.items():
        for song_ID in song_ID_list:
            dummy_song = list()
            artists.append(Artist2id[artist])
            song = ID_to_POS[song_ID]
            while len(song) != 100:
                song.append(["PADDING"])
            for line in song:
                while len(line) != 20:
                    line.append("PADDING")
            for line in song:
                #print("line is",line)
                POS_line = list()
                for tag in line:
                    if tag == "PADDING":
                        POS_line.append(0) # if the value is padding, then add 0
                    else:
                        POS_line.append(0.5) # if the value is any of the POS tags, then add the dummy value 0.5
                dummy_song.append(POS_line)
            songs.append(dummy_song)
    return songs, artists


# using the function, form the datasets in python list format
train_dummy_input_data, train_dummy_labels = dummy_gen(train_df)
dev_dummy_input_data, dev_dummy_labels = dummy_gen(dev_df)
test_dummy_input_data, test_dummy_labels = dummy_gen(test_df)


# in the end store these as pickle variables for later use
writePickle(train_dummy_input_data, "cnn_data_inputs/train_dummy_input_data")
writePickle(train_dummy_labels, "cnn_data_inputs/train_dummy_labels")
writePickle(dev_dummy_input_data, "cnn_data_inputs/dev_dummy_input_data")
writePickle(dev_dummy_labels, "cnn_data_inputs/dev_dummy_labels")
writePickle(test_dummy_input_data, "cnn_data_inputs/test_dummy_input_data")
writePickle(test_dummy_labels, "cnn_data_inputs/test_dummy_labels")

In [ ]:
print("An example of training dummy input data is:", train_dummy_input_data[0])
print("The first training dummy label is", train_dummy_labels[0])

#### Convert the generated input formats to CNN compatible format

In [ ]:
import numpy as np

train_dummy_input_data = np.array(train_dummy_input_data)
train_dummy_labels = np.array(train_dummy_labels)
dev_dummy_input_data = np.array(dev_dummy_input_data)
dev_dummy_labels = np.array(dev_dummy_labels)
test_dummy_input_data = np.array(test_dummy_input_data)
test_dummy_labels = np.array(test_dummy_labels)

train_dummy_input_data = train_dummy_input_data.astype('float32')
dev_dummy_input_data = dev_dummy_input_data.astype('float32')
test_dummy_input_data = test_dummy_input_data.astype('float32') 

In [ ]:
X_train_dummy = train_dummy_input_data.reshape(len(train_dummy_input_data),100,20,1)
X_dev_dummy = dev_dummy_input_data.reshape(len(dev_dummy_input_data),100,20,1)
X_test_dummy = test_dummy_input_data.reshape(len(test_dummy_input_data),100,20,1)

In [ ]:
from keras.utils import to_categorical

y_train_dummy = to_categorical(train_dummy_labels)
y_dev_dummy = to_categorical(dev_dummy_labels)
y_test_dummy = to_categorical(test_dummy_labels)

In [ ]:
# save the variables
writePickle(X_train_dummy,"cnn_data_inputs/dummy_Keras/X_train_dummy")
writePickle(X_dev_dummy,"cnn_data_inputs/dummy_Keras/X_dev_dummy")
writePickle(X_test_dummy,"cnn_data_inputs/dummy_Keras/X_test_dummy")
writePickle(y_train_dummy,"cnn_data_inputs/dummy_Keras/y_train_dummy")
writePickle(y_dev_dummy,"cnn_data_inputs/dummy_Keras/y_dev_dummy")
writePickle(y_test_dummy,"cnn_data_inputs/dummy_Keras/y_test_dummy")